In [1]:
# Basic
import torch
import math
import torch.nn as nn
from torch.optim.lr_scheduler import StepLR, ReduceLROnPlateau
import numpy as np
from numpy import random
torch.set_num_threads(4)

# PDEi
from utils_pde.utils_pde_2dpoisson import Poisson2D

# Viz
from utils_tools.utils_result_viz import plot_predictions_2D

# Base Mdoels
from utils_uqmd.utils_uq_dropout import DropoutPINN
from utils_uqmd.utils_uq_mlp import MLPPINN
from utils_uqmd.utils_uq_vi import VIBPINN
from utils_uqmd.utils_uq_distance import DistanceUQPINN

# CP
from utils_uqmd.utils_uq_cp import CP

# Ensure reproducibility
import random, numpy as np, torch
seed = 12345
random.seed(seed)
np.random.seed(seed)
torch.manual_seed(seed)
if torch.cuda.is_available():
    torch.cuda.manual_seed_all(seed)

# Data Noise
data_noise = 0.05

# Define PDE
domain=((0.0, 1.0), (0.0, 1.0))
true_solution=lambda xy: torch.sin(math.pi * xy[..., 0:1]) * torch.sin(math.pi * xy[..., 1:2])
pde = Poisson2D(domain, true_solution)

# Generate training and testing data of the poisson function
(X_train, Y_train) = pde.data_generation(500, data_noise)
(X_test, Y_test) = pde.data_generation(100, data_noise)
(X_calibration, Y_calibration) = pde.data_generation(200, data_noise)

# Number of colocation point
colloc_pt_num = 100

# Generating alphas to test
def generating_alphas(a_num):
    alphas = torch.linspace(0.0, 1.0, 10).view(-1, 1)
    return alphas[1:a_num-1]
alphas = generating_alphas(10)

# ────────────────────────────────────────────────────────────────────────────────
# 4. Build Dropout-PINN (1 input → 1 output)
# ────────────────────────────────────────────────────────────────────────────────
from utils_uqmd.utils_uq_dropout import DropoutPINN

model_args = {
    "pde_class":pde,
    "input_dim":2,
    "hidden_dims":[64, 128, 265, 265, 128, 64],
    "output_dim":1,
}

dist_pinn = DistanceUQPINN(**model_args)

# ────────────────────────────────────────────────────────────────────────────────
# 5. Hyper-parameter grid & CP settings
# ────────────────────────────────────────────────────────────────────────────────
fit_args = dict(coloc_pt_num=200, X_train=X_train, Y_train=Y_train)
fit_kwargs_grid = dict(
    epochs=[1000, 20000, 40000, 60000],
    λ_pde=[1.0, 2.0, 3.0, 5.0, 9.0], λ_ic=[5.0, 10.0], λ_data=[1.0, 2.50, 5.0],
    lr=[1e-3], stop_schedule=[20000]
)

baseline_pred_kwargs = dict(n_samples=200)

# Conformal-prediction settings
alphas = torch.linspace(0.01, 0.99, 20).view(-1, 1)

from utils_uqmd.utils_uq_cp import CP

# CP Model
cp_pred_kwargs = {
        "X_train":X_train,  "Y_train":Y_train,
        "X_cal":X_calibration, "Y_cal":Y_calibration,
        "heuristic_u":"raw_std",  # Change this based on cp
        "k":10
}

cp_testing_args = {
    "alphas":alphas, 
    "X_test":X_test, "Y_test":Y_test, 
    "X_cal":X_calibration, "Y_cal":Y_calibration, "X_train":X_train, "Y_train":Y_train, 
    "heuristic_u":"raw_std", # Change base on if the baseline cp
    "k":10
}

baseline_testing_args = { 
    "uqmodel":dist_pinn,   # Change this
    "alphas":alphas, 
    "X_test":X_test, "Y_test":Y_test,
    "heuristic_u":"feature",
    "n_samples":200, 
}

# --------------------------------------------
# Defining Plotting Grid
# --------------------------------------------
n_grid = 100
x = torch.linspace(0, 1, n_grid)
y = torch.linspace(0, 1, n_grid)
X, Y = torch.meshgrid(x, y, indexing='xy')
grid_test = torch.cat([X.reshape(-1, 1), Y.reshape(-1, 1)], dim=1)



# ────────────────────────────────────────────────────────────────────────────────
# 6. Launch tuning with the NEW 1-D coverage plot
# ────────────────────────────────────────────────────────────────────────────────
from utils_tools.utils_tuning       import hyperparameter_tuning
from utils_tools.utils_result_viz   import plot_1D_comparison_with_coverage, plot_2D_comparison_with_coverage
from utils_tools.utils_result_metrics import dist_test_uncertainties

hyperparameter_tuning(
    plot_title="Distance CP Model", # Change this
    # Model Fitting & Predicting
    uqmodel=dist_pinn,  # Change this|
    alpha=0.05, 
    X_test=grid_test, Y_test=Y_test, 
    fit_args=fit_args, fit_kwargs_grid=fit_kwargs_grid, baseline_pred_kwargs=baseline_pred_kwargs, cp_pred_kwargs=cp_pred_kwargs, 
    true_solution=pde.true_solution,
    # Coverage Test
    baseline_testing_args=baseline_testing_args, cp_testing_args=cp_testing_args,
    baseline_test_uncertainties=dist_test_uncertainties, # Change this
    # Plotting function
    plotting_func=plot_2D_comparison_with_coverage, # 
    save_dir="2dpoisson_dist_cp", # Change this
    X_vis=X_train, Y_vis=Y_train,
    X_validation=X_test, Y_validation=Y_test
)

Using device: cpu
Using device: cpu

[🔎] Trying: {'epochs': 1000, 'λ_pde': 1.0, 'λ_ic': 5.0, 'λ_data': 1.0, 'lr': 0.001, 'stop_schedule': 20000}

[🟠] Training...
ep     1 | L=9.07e+01 | lr=1.0e-03
ep   500 | L=1.35e-01 | lr=1.0e-03
ep  1000 | L=6.28e-02 | lr=1.0e-03

[🟠] Inferencing...

[🟠] Computing Coverage...


100%|██████████| 20/20 [00:00<00:00, 228.95it/s]



[✅] Data Loss = 3.744e-03

[🔎] Trying: {'epochs': 1000, 'λ_pde': 1.0, 'λ_ic': 5.0, 'λ_data': 2.5, 'lr': 0.001, 'stop_schedule': 20000}

[🟠] Training...
ep     1 | L=7.06e-02 | lr=1.0e-03
ep   500 | L=6.91e-02 | lr=1.0e-03


KeyboardInterrupt: 

<Figure size 640x480 with 0 Axes>